In [ ]:
%pip install langchain langchain-openai

Tools can be just about anything — APIs, functions, databases, etc. Tools allow us to extend the capabilities of a model beyond just outputting text/messages

In [17]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool

import os
from dotenv import load_dotenv

load_dotenv(override=True)
#os.environ["OPENAI_API_KEY"] = "your key"


@tool
def multiply(first_int: int, second_int: int) -> int:
    """Multiply two integers together."""
    return first_int * second_int

@tool
def divide(first_int: int, second_int: int) -> int:
    """Divide first integer by second integer."""
    return first_int / second_int

@tool
def add(first_int: int, second_int: int) -> int:
    "Add two integers."
    return first_int + second_int

@tool
def subtract(first_int: int, second_int: int) -> int:
    "sbutract two integers."
    return first_int - second_int


#multiply.invoke({"first_int": 4, "second_int": 5})
#divide.invoke({"first_int": 4, "second_int": 2})
#add.invoke({"first_int": 4, "second_int": 5})
subtract.invoke({"first_int": 4, "second_int": 5})


-1

In [ ]:

from operator import itemgetter
from typing import Union

from langchain.output_parsers import JsonOutputToolsParser
from langchain_core.runnables import (
    Runnable,
    RunnableLambda,
    RunnableMap,
    RunnablePassthrough,
)

model = ChatOpenAI(openai_api_key=os.environ["OPENAI_API_KEY"], model_name="gpt-4-0125-preview")

tools = [multiply, divide, add, subtract]
model_with_tools = model.bind_tools(tools)
model_with_tools.kwargs["tools"]


In [22]:
tool_map = {tool.name: tool for tool in tools}
print(tool_map)

{'multiply': StructuredTool(name='multiply', description='multiply(first_int: int, second_int: int) -> int - Multiply two integers together.', args_schema=<class 'pydantic.v1.main.multiplySchemaSchema'>, func=<function multiply at 0x0000017F736EDB80>), 'divide': StructuredTool(name='divide', description='divide(first_int: int, second_int: int) -> int - Divide first integer by second integer.', args_schema=<class 'pydantic.v1.main.divideSchemaSchema'>, func=<function divide at 0x0000017F717BAB80>), 'add': StructuredTool(name='add', description='add(first_int: int, second_int: int) -> int - Add two integers.', args_schema=<class 'pydantic.v1.main.addSchemaSchema'>, func=<function add at 0x0000017F736ED310>), 'subtract': StructuredTool(name='subtract', description='subtract(first_int: int, second_int: int) -> int - sbutract two integers.', args_schema=<class 'pydantic.v1.main.subtractSchemaSchema'>, func=<function subtract at 0x0000017F5882F3A0>)}



JsonOutputToolsParser, a built-in LangChain output parser that converts an OpenAI function-calling response to a list of {"type": "TOOL_NAME", "args": {...}} dicts with the tools to invoke and arguments to invoke them with.

In [23]:
from langchain.output_parsers import JsonOutputToolsParser

#chain = model | StrOutputParser()
chain = model_with_tools | JsonOutputToolsParser()
chain.invoke("What's four times 23")


[{'type': 'multiply', 'args': {'first_int': 4, 'second_int': 23}}]

In [34]:
def call_tool(tool_invocation: dict) -> Union[str, Runnable]:
    """Function for dynamically constructing the end of the chain based on the model-selected tool."""
    tool = tool_map[tool_invocation["type"]]
    return RunnablePassthrough.assign(output=itemgetter("args") | tool)

# .map() allows us to apply a function to a list of inputs.
call_tool_list = RunnableLambda(call_tool).map()
chain = model_with_tools | JsonOutputToolsParser() | call_tool_list

#chain.invoke("What's four times 23")
chain.invoke("What's 8 divided by 2?")
#chain.invoke("What's 8 divided by 2?")[0]['output']


[{'type': 'divide', 'args': {'first_int': 8, 'second_int': 2}, 'output': 4.0}]